In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functions import transformations

In [28]:
v_clean = pd.read_csv("./vehicles_cleaned_train.csv")
display(v_clean.head())
v_clean_norm = transformations(v_clean) #Por enquanto a normalização está comentada! (Conversar depois sobre)
print(v_clean_norm.shape)

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,10400,2011,excellent,4 cylinders,gas,81300,automatic,mid-size,sedan
1,6900,2007,excellent,6 cylinders,gas,79000,automatic,full-size,sedan
2,18900,2018,like new,4 cylinders,gas,5000,automatic,full-size,sedan
3,6000,2010,excellent,4 cylinders,gas,97600,automatic,mid-size,hatchback
4,19995,2013,good,6 cylinders,gas,95782,automatic,mid-size,sedan


(8338, 25)


In [35]:
# Column vector with prices
d = v_clean_norm[:,0]

# Data matrix (all features except prices)
C = v_clean_norm[:, 1:]

# Get the number of columns
M = C.shape[1]

# Get the number of lines
N = d.shape[0]

# Create a matrix with N lines and M+2 columns filled with zeros
# We must have M+2 columns to fit the bias and the year squared
X = np.ones((N, M+2)) 
print(X.shape)

(8338, 26)


In [36]:
X[:,1] = C[:, 0] - 2001 # Put the year into the matrix (2001 is the reference, discuss later)
X[:,2] = (C[:, 0] - 2001)**2 # Put the year squared into the matrix
X[:,3:] = C[:, 1:] # Put the odometer and the category into the matrix

print(X)

[[  1.  10. 100. ...   0.   0.   0.]
 [  1.   6.  36. ...   0.   0.   0.]
 [  1.  17. 289. ...   0.   0.   0.]
 ...
 [  1.   2.   4. ...   0.   0.   0.]
 [  1.  15. 225. ...   0.   0.   0.]
 [  1.  17. 289. ...   0.   0.   0.]]


In [37]:
R = X.T @ X
p = X.T @ d
wo = np.linalg.solve(R,p)
e = d - X @ wo

In [38]:
# Load test dataset
v_clean_test = pd.read_csv("./vehicles_cleaned_test.csv") 
display(v_clean_test)

# Transform the dataset (the return is a numpy now)
v_clean_norm_test = transformations(v_clean_test)
print(v_clean_norm_test.shape)

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,11995,2018,excellent,4 cylinders,gas,19000,automatic,mid-size,sedan
1,20500,2017,excellent,6 cylinders,gas,68000,automatic,full-size,sedan
2,15800,2018,like new,4 cylinders,gas,41200,automatic,full-size,sedan
3,20980,2015,excellent,6 cylinders,gas,82845,automatic,mid-size,sedan
4,13999,2019,like new,4 cylinders,gas,29638,automatic,full-size,sedan
...,...,...,...,...,...,...,...,...,...
2079,15895,2017,like new,4 cylinders,gas,33734,automatic,mid-size,sedan
2080,14390,2017,excellent,4 cylinders,gas,71219,automatic,mid-size,sedan
2081,17995,2018,like new,4 cylinders,gas,37000,automatic,mid-size,sedan
2082,8900,2011,like new,6 cylinders,gas,45346,automatic,full-size,sedan


(2084, 25)


In [33]:
# Get the prices from the test dataset
prices_test = v_clean_norm_test[:, 0]

# Turn the weights into a column vector
wo = wo.reshape(-1,1)

# Create the matrix that represents the dataset
# Note that the number of columns is equal to the number of weights
data = np.ones((prices_test.shape[0], wo.shape[0]))

# Insert the years into the matrix
data[:,1] = v_clean_norm_test[:,1] - 2001

# Insert the years squared
data[:,2] = (v_clean_norm_test[:,1] - 2001)**2

# Insert the rest of the data (odometer and the dummies)
data[:,3:] = v_clean_norm_test[:,2:]

# Create column vector to receive the result
prices_pred = np.zeros((data.shape[0], 1))

# Predict the price using the model
# Note that it's a AX=B multiplication, where A is the data, X is the weights and B is the price
prices_pred = data @ wo

[[  1.  17. 289. ...   0.   0.   0.]
 [  1.  16. 256. ...   0.   0.   0.]
 [  1.  17. 289. ...   0.   0.   0.]
 ...
 [  1.  17. 289. ...   0.   0.   0.]
 [  1.  10. 100. ...   0.   0.   0.]
 [  1.  13. 169. ...   0.   0.   0.]]


In [34]:
error = abs(prices_pred.reshape(-1,1) - prices_test.reshape(-1,1))
error_squared = np.mean(error**2)

print(error_squared) # ???? EXTREMELY HIGH! I will see it later

27365888.02608643
